In [ ]:
import os
from IPython.display import Image, display
from tensorflow.keras.preprocessing.image import load_img
import PIL
from PIL import ImageOps
import re
import numpy as np
import tensorflow as tf
import keras
from keras import layers
from keras.layers import MaxPooling3D, \
    Input, ConvLSTM2D, UpSampling2D, \
    MaxPooling2D, Conv2D, Concatenate, Conv3D, \
    Dropout, BatchNormalization, Add, MaxPool2D, Conv2DTranspose, MaxPool3D, Conv3DTranspose, UpSampling3D, \
    concatenate, MaxPooling3D, Bidirectional, TimeDistributed
from tensorflow.python.client import device_lib
import random
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
from keras import backend as K
import neptune
from keras.callbacks import Callback
from sklearn.model_selection import train_test_split


In [ ]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

os.chdir('/home/aijjeh/Desktop/Phd_Projects/PC_uint_cell_dispersion_curves/dataset/')
X_train = np.load('train_x.npy')
Y_train = np.load('train_y_mat.npy')
Y_train = (Y_train[:,:,0] / 5.2667e5)

# x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.07, random_state=42)

x_train = X_train[:450]
y_train = Y_train[:450]


x_test = X_train[450:]
y_test = Y_train[450:]

# batches = 32
# train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batches)
# # val_set = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(batches)
# test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batches)

In [ ]:
def conv_block(in_blk, num):
    layer11 = tf.keras.layers.Conv1D(num, 3,
                                     padding='same',
                                     activation='relu')(in_blk)
    layer12 = tf.keras.layers.Conv1D(num, 3,
                                     padding='same',
                                     activation='relu')(layer11)
    return tf.keras.layers.concatenate([layer11, layer12], axis=-1)

In [ ]:
filter_size = 16
img_shape = (256, 256 ,1)
epochs= 3000

In [ ]:
########################################################################################################################
# First CNN model
########################################################################################################################
# Encoder
########################################################################################################################
inputs_1 = tf.keras.Input(shape=img_shape)
blk1 = conv_block(inputs_1, filter_size)

DS1 = tf.keras.layers.MaxPool2D((2, 2))(blk1)

blk2 = conv_block(DS1, filter_size * 2)

DS2 = tf.keras.layers.MaxPool2D((2, 2))(blk2)

blk3 = conv_block(DS2, filter_size * 4)

DS3 = tf.keras.layers.MaxPool2D((2, 2))(blk3)

blk4 = conv_block(DS3, filter_size * 8)

DS4 = tf.keras.layers.MaxPool2D((2, 2))(blk4)

blk5 = conv_block(DS4, filter_size * 8)

DS5 = tf.keras.layers.MaxPool2D((2, 2))(blk5)

blk6 = conv_block(DS5, filter_size * 8)

DS6 = tf.keras.layers.MaxPool2D((2, 2))(blk6)

In [ ]:
########################################################################################################################
# BottleNeck
########################################################################################################################
bottleneck = conv_block(DS6, filter_size * 8)
bottleneck = tf.keras.layers.Flatten()(bottleneck)
print(tf.shape(bottleneck))
output1 = tf.keras.layers.Dense(4096, activation='relu')(bottleneck)
# output1 = tf.keras.layers.Dropout(0.4)(output1)
output1 = tf.keras.layers.Dense(4096, activation='relu')(output1)
output1 = tf.keras.layers.Dropout(0.5)(output1)
output1 = tf.keras.layers.Dense(4096, activation='relu')(output1)
output1 = tf.keras.layers.Dropout(0.5)(output1)
output1 = tf.keras.layers.Dense(1464, activation='relu')(output1)

output1 = tf.keras.layers.Reshape((1464, 1))(output1)


In [ ]:
########################################################################################################################
# Decoder
########################################################################################################################
# UP1 = tf.keras.layers.UpSampling2D((2, 2))(bottleneck)
# blk5 = conv_block(UP1, filter_size * 8)
# UP2 = tf.keras.layers.UpSampling2D((2, 1))(blk5)
# blk6 = conv_block(UP2, filter_size * 4)
# UP3 = tf.keras.layers.UpSampling2D((2, 1))(blk6)
# blk7 = conv_block(UP3, filter_size * 2)
# UP4 = tf.keras.layers.UpSampling2D((2, 1))(blk7)
# blk8 = conv_block(UP4, filter_size)

In [ ]:
########################################################################################################################
# Output layer
########################################################################################################################
cnn_model = tf.keras.models.Model(inputs=inputs_1, outputs=output1)
cnn_model.summary()

In [ ]:
########################################################################################################################
# Model compile and training
########################################################################################################################
cnn_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4),
                  loss=tf.keras.losses.MeanSquaredError(),
                  metrics=[tf.keras.metrics.MeanSquaredError(),
                          tf.keras.metrics.CosineSimilarity()],)

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2000, min_delta=1e-6)
cnn_model.fit(x=x_train, 
              y=y_train,
              batch_size=16,
              validation_split=0.1,
              epochs=epochs,             
              callbacks=[callback])

In [ ]:
os.chdir('/home/aijjeh/Desktop/Phd_Projects/PC_uint_cell_dispersion_curves/h5_models/')
cnn_model.save("VAE_ANN_PC_uint_cell_ver_5.h5")